# IMPORT :

In [1]:
import sys
import os
import re
import gensim.downloader as api
import nltk
import numpy as np
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from symspellpy import SymSpell, Verbosity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# UTILS FUNCTIONS :

In [2]:
# Function to compute the average word vector for a tweet
def get_avg_embedding(tweet, model, vector_size=200):
    words = tweet.split()  # Tokenize by whitespace
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:  # If no words in the tweet are in the vocabulary, return a zero vector
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)


# Basic preprocessing function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenization
    words = text.split()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# PREPROCCESS PART 0 :

In [3]:
print("PREPROCESS PART 0...")
sys.stdout.flush()


os.makedirs("tmp/", exist_ok = True)


# Download some NLP models for processing, optional
nltk.download('stopwords')
nltk.download('wordnet')

# Load GloVe model with Gensim's API
embeddings_model = api.load("glove-twitter-200")  # 200-dimensional GloVe embeddings


print("PREPROCESS PART 0 : OK")
sys.stdout.flush()

PREPROCESS PART 0...


[nltk_data] Downloading package stopwords to /home/hbar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hbar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


PREPROCESS PART 0 : OK


# PREPROCESS PART 1 :

In [4]:
print("PREPROCESS PART 1...")
sys.stdout.flush()


if not os.path.isfile("tmp/processing1.csv"):
    # Read all training files and concatenate them into one dataframe
    li = []
    for filename in os.listdir("train_tweets"):
        df = pd.read_csv("train_tweets/" + filename)
        li.append(df)
    df = pd.concat(li, ignore_index=True)

    # Apply preprocessing to each tweet
    df['Tweet'] = df['Tweet'].apply(preprocess_text)

    df.to_csv("tmp/processing1.csv", index=False, encoding="utf-8")
else:
    df = pd.read_csv("tmp/processing1.csv")


print("PREPROCESS PART 1 : OK")
sys.stdout.flush()

PREPROCESS PART 1...
PREPROCESS PART 1 : OK


# PREPROCESS PART 2 :

In [5]:
print("PREPROCESS PART 2...")
sys.stdout.flush()


vector_size = 200  # Adjust based on the chosen GloVe model


if not os.path.isfile("tmp/X.npy") and not os.path.isfile("tmp/y.npy"):
    # Apply preprocessing to each tweet and obtain vectors
    tweet_vectors = np.vstack([get_avg_embedding(tweet, embeddings_model, vector_size) for tweet in df['Tweet']])
    tweet_df = pd.DataFrame(tweet_vectors)

    # Attach the vectors into the original dataframe
    period_features = pd.concat([df, tweet_df], axis=1)
    # Drop the columns that are not useful anymore
    period_features = period_features.drop(columns=['Timestamp', 'Tweet'])
    # Group the tweets into their corresponding periods. This way we generate an average embedding vector for each period
    period_features = period_features.groupby(['MatchID', 'PeriodID', 'ID']).mean().reset_index()

    # We drop the non-numerical features and keep the embeddings values for each period
    X = period_features.drop(columns=['EventType', 'MatchID', 'PeriodID', 'ID']).values
    # We extract the labels of our training samples
    y = period_features['EventType'].values

    np.save("tmp/X.npy", X)
    np.save("tmp/y.npy", y)
else:
    X = np.load("tmp/X.npy")
    y = np.load("tmp/y.npy")


print("PREPROCESS PART 2 : OK")
sys.stdout.flush()

PREPROCESS PART 2...
PREPROCESS PART 2 : OK


# Evaluating on a test set :

In [ ]:
# We split our data into a training and test set that we can use to train our classifier without fine-tuning into the
# validation set and without submitting too many times into Kaggle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test set: ", accuracy_score(y_test, y_pred))

In [ ]:
clf = SVC(kernel="rbf", random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test set: ", accuracy_score(y_test, y_pred))

In [ ]:
clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test set: ", accuracy_score(y_test, y_pred))

clf = RandomForestClassifier(random_state=42, n_estimators=170, max_depth=10, max_features='sqrt')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test set: ", accuracy_score(y_test, y_pred))

In [ ]:
clf = XGBClassifier(random_state=42, n_estimators=100, eval_metric="logloss")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test set: ", accuracy_score(y_test, y_pred))

clf = XGBClassifier(random_state=42, n_estimators=170, learning_rate=0.1, max_depth=6, subsample=1, eval_metric="logloss", booster="gbtree")
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test set: ", accuracy_score(y_test, y_pred))

# Combine Evaluation on a test set :

In [ ]:
# Modèles de base
rf = RandomForestClassifier(random_state=42, n_estimators=100)
xgb = XGBClassifier(random_state=42, n_estimators=100, eval_metric="logloss")
base_models = [ ('rf', rf), ('xgb', xgb) ]

# Méta-modèle
meta_model = LogisticRegression()

# Stacking
stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=3)
stack.fit(X_train, y_train)

# Évaluation
stack_pred = stack.predict(X_test)
print("Accuracy (Stacking):", accuracy_score(y_test, stack_pred))

In [ ]:
# Modèles de base
rf = RandomForestClassifier(random_state=42, n_estimators=100)
xgb = XGBClassifier(random_state=42, n_estimators=100, eval_metric="logloss")
lr = LogisticRegression(random_state=42, max_iter=1000)
svc =  SVC(kernel="rbf", random_state=42)
base_models = [ ('rf', rf), ('xgb', xgb), ('lr', lr), ('svc', svc) ]

# Méta-modèle
meta_model = LogisticRegression()

# Stacking
stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=3)
stack.fit(X_train, y_train)

# Évaluation
stack_pred = stack.predict(X_test)
print("Accuracy (Stacking):", accuracy_score(y_test, stack_pred))

In [ ]:
# Modèles de base
rf =  RandomForestClassifier(random_state=42, n_estimators=170, max_depth=10, max_features='sqrt')
xgb = XGBClassifier(random_state=42, n_estimators=170, learning_rate=0.1, max_depth=6, subsample=1, eval_metric="logloss", booster="gbtree")
base_models = [ ('rf', rf), ('xgb', xgb) ]

# Méta-modèle
meta_model = LogisticRegression()

# Stacking
stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=3)
stack.fit(X_train, y_train)

# Évaluation
stack_pred = stack.predict(X_test)
print("Accuracy (Stacking):", accuracy_score(y_test, stack_pred))

In [ ]:
# Modèles de base
rf =  RandomForestClassifier(random_state=42, n_estimators=170, max_depth=10, max_features='sqrt')
xgb = XGBClassifier(random_state=42, n_estimators=170, learning_rate=0.1, max_depth=6, subsample=1, eval_metric="logloss", booster="gbtree")
lr = LogisticRegression(random_state=42, max_iter=1000)
svc =  SVC(kernel="rbf", random_state=42)
base_models = [ ('rf', rf), ('xgb', xgb), ('lr', lr), ('svc', svc) ]

# Méta-modèle
meta_model = LogisticRegression()

# Stacking
stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=3)
stack.fit(X_train, y_train)

# Évaluation
stack_pred = stack.predict(X_test)
print("Accuracy (Stacking):", accuracy_score(y_test, stack_pred))

## Find the best cv value :

In [ ]:
# Modèles de base
rf = RandomForestClassifier(random_state=42, n_estimators=100)
xgb = XGBClassifier(random_state=42, n_estimators=100, eval_metric="logloss")
base_models = [ ('rf', rf), ('xgb', xgb) ]
    
# Méta-modèle
meta_model = LogisticRegression()

# Find best cv
max_score = 0
max_cv = 0

for i in range(2, 21):
    # Stacking
    stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=i)
    stack.fit(X_train, y_train)
    
    # Évaluation
    stack_pred = stack.predict(X_test)
    accuracy = accuracy_score(y_test, stack_pred)

    print(accuracy)

    # Udapte ?
    if max_score < accuracy:
        max_score = accuracy
        max_cv = i

# Print result
print("Accuracy (Stacking):", max_score, "with cv =", max_cv)

In [ ]:
# Modèles de base
rf = RandomForestClassifier(random_state=42, n_estimators=100)
xgb = XGBClassifier(random_state=42, n_estimators=100, eval_metric="logloss")
lr = LogisticRegression(random_state=42, max_iter=1000)
svc =  SVC(kernel="rbf", random_state=42)
base_models = [ ('rf', rf), ('xgb', xgb), ('lr', lr), ('svc', svc) ]
    
# Méta-modèle
meta_model = LogisticRegression()

# Find best cv
max_score = 0
max_cv = 0

for i in range(2, 21):
    # Stacking
    stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=i)
    stack.fit(X_train, y_train)
    
    # Évaluation
    stack_pred = stack.predict(X_test)
    accuracy = accuracy_score(y_test, stack_pred)

    print(accuracy)

    # Udapte ?
    if max_score < accuracy:
        max_score = accuracy
        max_cv = i

# Print result
print("Accuracy (Stacking):", max_score, "with cv =", max_cv)

In [ ]:
# Modèles de base
rf =  RandomForestClassifier(random_state=42, n_estimators=170, max_depth=10, max_features='sqrt')
xgb = XGBClassifier(random_state=42, n_estimators=170, learning_rate=0.1, max_depth=6, subsample=1, eval_metric="logloss", booster="gbtree")
base_models = [ ('rf', rf), ('xgb', xgb) ]
    
# Méta-modèle
meta_model = LogisticRegression()

# Find best cv
max_score = 0
max_cv = 0

for i in range(2, 21):
    # Stacking
    stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=i)
    stack.fit(X_train, y_train)
    
    # Évaluation
    stack_pred = stack.predict(X_test)
    accuracy = accuracy_score(y_test, stack_pred)

    print(accuracy)

    # Udapte ?
    if max_score < accuracy:
        max_score = accuracy
        max_cv = i

# Print result
print("Accuracy (Stacking):", max_score, "with cv =", max_cv)

In [ ]:
# Modèles de base
rf =  RandomForestClassifier(random_state=42, n_estimators=170, max_depth=10, max_features='sqrt')
xgb = XGBClassifier(random_state=42, n_estimators=170, learning_rate=0.1, max_depth=6, subsample=1, eval_metric="logloss", booster="gbtree")
lr = LogisticRegression(random_state=42, max_iter=1000)
svc =  SVC(kernel="rbf", random_state=42)
base_models = [ ('rf', rf), ('xgb', xgb), ('lr', lr), ('svc', svc) ]
    
# Méta-modèle
meta_model = LogisticRegression()

# Find best cv
max_score = 0
max_cv = 0

for i in range(2, 21):
    # Stacking
    stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=i)
    stack.fit(X_train, y_train)
    
    # Évaluation
    stack_pred = stack.predict(X_test)
    accuracy = accuracy_score(y_test, stack_pred)

    print(accuracy)

    # Udapte ?
    if max_score < accuracy:
        max_score = accuracy
        max_cv = i

# Print result
print("Accuracy (Stacking):", max_score, "with cv =", max_cv)

# For Kaggle submission :

In [14]:
print("KAGGLE...")
sys.stdout.flush()


# This time we train our classifier on the full dataset that it is available to us.

rf =  RandomForestClassifier(random_state=42, n_estimators=170, max_depth=10, max_features='sqrt')
xgb = XGBClassifier(random_state=42, n_estimators=170, learning_rate=0.1, max_depth=6, subsample=1, eval_metric="logloss", booster="gbtree")
lr = LogisticRegression(random_state=42, max_iter=1000)
svc = SVC(kernel="rbf", random_state=42)

base_models = [ ('rf', rf), ('xgb', xgb), ('lr', lr), ('svc', svc) ]
meta_model = LogisticRegression()
clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=3)

clf.fit(X, y)
predictions = []

print(X)
print(X.shape[1])


# We read each file separately, we preprocess the tweets and then use the classifier to predict the labels.
# Finally, we concatenate all predictions into a list that will eventually be concatenated and exported
# to be submitted on Kaggle.
for fname in os.listdir("eval_tweets"):
    val_df = pd.read_csv("eval_tweets/" + fname)
    val_df['Tweet'] = val_df['Tweet'].apply(preprocess_text)

    tweet_vectors = np.vstack([get_avg_embedding(tweet, embeddings_model, vector_size) for tweet in val_df['Tweet']])
    tweet_df = pd.DataFrame(tweet_vectors)

    period_features = pd.concat([df, tweet_df], axis=1)
    period_features = period_features.drop(columns=['Timestamp', 'Tweet'])
    period_features = period_features.groupby(['MatchID', 'PeriodID', 'ID']).mean().reset_index()
    
    X_pred = period_features.drop(columns=['MatchID', 'PeriodID', 'ID']).values

    print(X_pred)
    print(X_pred.shape[1])
    
    preds = clf.predict(X_pred)
    
    period_features['EventType'] = preds
    predictions.append(period_features[['ID', 'EventType']])

pred_df = pd.concat(predictions)
pred_df.to_csv('predictions.csv', index=False)


print("KAGGLE : OK")
sys.stdout.flush()

KAGGLE...
[[ 0.14270035  0.27335462  0.02405166 ...  0.09354731 -0.00983074
   0.14555588]
 [ 0.15579546  0.27469747  0.02270712 ...  0.100027   -0.00811471
   0.1272895 ]
 [ 0.14934901  0.29195836  0.03574221 ...  0.08154248 -0.00828414
   0.13079854]
 ...
 [ 0.04854218  0.27198784  0.05176172 ...  0.13164098 -0.06986898
   0.17400647]
 [ 0.05180092  0.26948526  0.05175036 ...  0.12463561 -0.07782839
   0.17300791]
 [ 0.04112091  0.22421245  0.08582967 ...  0.1013057  -0.10145353
   0.17451022]]
200
[[ 0. nan nan ... nan nan nan]
 [ 0. nan nan ... nan nan nan]
 [ 0. nan nan ... nan nan nan]
 ...
 [ 1. nan nan ... nan nan nan]
 [ 1. nan nan ... nan nan nan]
 [ 1. nan nan ... nan nan nan]]
201


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values